In [ ]:
import csv

def parse_txt(filename, oriented=True):
    """
    Parse data from txt file into dict python type.
    JSON serializable.
    """
    data = {}
    with open(filename) as file:
        
        line = file.readline()
        while line:
            
            # skip comments
            if line[0] == '#':
                line = file.readline()
                continue
            
            parent, child = line.split()
            parent = int(parent)
            child = int(child)
            
            # rows in data file can be duplicated
            if parent in data:
                if child not in data[parent]['linked']:
                    data[parent]['linked'].append(child)
                    data[parent]['degree'] += 1
            else:
                data[parent] = { 
                    'linked': [child],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            if oriented:
                if child not in data:
                    data[child] = { 
                    'linked': [],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            else:
                if child in data:
                    if parent not in data[child]['linked']:
                        data[child]['linked'].append(parent)
                        data[child]['degree'] += 1

                else:    
                    data[child] = {
                        'linked': [parent],
                        'distances': {},
                        'degree': 1,
                        'centrality': 0,
                        'marked': False,
                        'active': True
                    }

            line = file.readline()

    return data

def parse_csv(filename, oriented=True):
    data = {}
    
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        
        for row in reader:
            
            parent = int(row[0])
            child = int(row[1])
            
            if parent in data:
                if child not in data[parent]['linked']:
                    data[parent]['linked'].append(child)
                    data[parent]['degree'] += 1
            else:
                data[parent] = { 
                    'linked': [child],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            if oriented:
                if child not in data:
                    data[child] = { 
                    'linked': [],
                    'distances': {},
                    'degree': 1,
                    'centrality': 0,
                    'marked': False,
                    'active': True
                }
                
            else:
                if child in data:
                    if parent not in data[child]['linked']:
                        data[child]['linked'].append(parent)
                        data[child]['degree'] += 1

                else:    
                    data[child] = {
                        'linked': [parent],
                        'distances': {},
                        'degree': 1,
                        'centrality': 0,
                        'marked': False,
                        'active': True
                    }
                    
    return data

def parse(filename, oriented=True):
    if filename.split('.')[-1] == 'txt':
        return parse_txt(filename, oriented)
    elif filename.split('.')[-1] == 'csv':
        return parse_csv(filename, oriented)

In [ ]:
FILENAME = 'test-google.txt'
ORIENTED = False

In [ ]:
data = parse(FILENAME, ORIENTED)
print(data)

In [ ]:
def count_vertices(graph):
    return len(graph)

def count_edges(graph):
    edges = 0
    for item in graph.values():
        edges += item['degree']
    return edges / 2

vertices = count_vertices(data)
edges = count_edges(data)
complete_graph_edges = vertices * (vertices - 1) / 2

print(f'Number of vertices in {FILENAME}: {vertices}')
print(f'Number of edges in {FILENAME}: {edges}')
print(f'Number of edges in complete graph: {complete_graph_edges}')
print(f'Density: {edges / complete_graph_edges}')

In [ ]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    for next in set(graph[start]["linked"]) - visited:
        dfs(graph, next, visited)
    return visited

def get_weak_connectivity_components(graph):
    weak_connectivity_components = []
    for item in graph.keys():
        component = dfs(graph,item)
        if component not in weak_connectivity_components:
            weak_connectivity_components.append(component)
    return weak_connectivity_components

def get_max_weak_connectivity_component_size(components):
    maximum = 0
    result = components[0]
    for item in components:
        if len(item) > maximum:
            maximum = len(item)
            result = item
            
    return (maximum, result)

weak_connectivity_components = get_weak_connectivity_components(data)
[max_weak_connectivity_component_size, max_weak_connectivity_component] = get_max_weak_connectivity_component_size(weak_connectivity_components)
print(f'Number of weak connectivity components in {FILENAME}: {len(weak_connectivity_components)}')
print(f'Proportion of vertices in max weak connectivity component: {max_weak_connectivity_component_size/vertices}')

In [ ]:
from collections import defaultdict

class Graph:
   
    def __init__(self,vertices,FILENAME=None):
        self.V = vertices
        self.graph = defaultdict(list) 
        if FILENAME:
            self.parse(FILENAME)
        
    def parse(self,FILENAME):
        with open(FILENAME) as file:
        
            line = file.readline()
            while line:

                if line[0] == '#':
                    line = file.readline()
                    continue

                parent, child = line.split()
                parent = int(parent)
                child = int(child)

                self.addEdge(parent,child)

                if parent in self.graph.keys():
                    if child not in self.graph[parent]:
                        self.graph[parent].append(child)
                else:
                    self.graph[parent] = [child]    

                if child not in self.graph.keys():
                    self.graph[child] = []

                line = file.readline()
    
    def addEdge(self,u,v):
        self.graph[u].append(v)
   
    def DFSUtil(self,v,visited,sublist):
        visited[v] = True
        sublist.append(v)
        for i in self.graph[v]:
            if visited[i] == False:
                self.DFSUtil(i,visited,sublist)
        return sublist
  
    def fillOrder(self,v,visited, stack):
        visited[v] = True
        for i in self.graph[v]:
            if visited[i] == False:
                self.fillOrder(i, visited, stack)
        stack = stack.append(v)
      
    def getTranspose(self):
        g = Graph(self.V)
        
        for i in self.graph:
            for j in self.graph[i]:
                g.addEdge(j,i)
        return g
  
    def printSCCs(self):
        result = []
        stack = []
        visited = dict.fromkeys(self.graph.keys(),False)
        for i in visited.keys():
            if visited[i] == False:
                self.fillOrder(i, visited, stack)
                
        gr = self.getTranspose()
        visited = dict.fromkeys(self.graph.keys(),False)
  
        while stack:
             i = stack.pop()
             if visited[i] == False:
                result.append(gr.DFSUtil(i, visited,[]))
        return result
                

graph = Graph(vertices,FILENAME)


In [ ]:
def get_max_strong_connectivity_component_size(components):
    maximum = 0
    for item in components:
        if len(item) > maximum:
            maximum = len(item)
    return maximum

strong_connectivity_components = graph.printSCCs()
max_strong_connectivity_component_size = get_max_strong_connectivity_component_size(strong_connectivity_components)
print(f'Number of strong connectivity components in {FILENAME}: {len(strong_connectivity_components)}')
print(f'Proportion of vertices in max strong connectivity component: {max_strong_connectivity_component_size/vertices}')

In [93]:
from random import randint

def bfs(graph, node,length_max): 
    visited = {}
    queue = []
    visited_nodes = []
    length = 0
    queue.append(node)
    while queue and length < length_max:          
        m = queue.pop(0) 
        visited[m] = []
        length += 1
        visited_nodes.append(m)
        for neighbour in graph[m]['linked']:
            visited[m].append(neighbour)
            length += 1
            if neighbour not in visited_nodes:
                visited_nodes.append(neighbour)
                queue.append(neighbour)
    return visited
    



def subgraph_induced_by_nodes(graph,nodes):
    start_node = list(nodes)[randint(0,len(nodes))]
    result = {}
    for node in nodes:
        result[node] = graph[node]
        for i in result[node]['linked']:
            if i not in nodes:
                result[node]['linked'].remove(i)
    return bfs(result, start_node, 500)

In [94]:
import queue
 
def minEdgeBFS(graph, u, v):
    visited = dict.fromkeys(graph.keys(),False)
    distance = dict.fromkeys(graph.keys(),0)
 
    # queue to do BFS.
    Q = queue.Queue()
    Q.put(u)
    visited[u] = True
    while (not Q.empty()):
        x = Q.get()
         
        for i in graph[x]:
            if visited[i]:
                continue
 
            distance[i] = distance[x] + 1
            Q.put(i)
            visited[i] = True
    return distance[v]
 
def distances(graph):
    distances = dict.fromkeys(graph.keys(),{})
    for i in graph.keys():
        for j in graph.keys():
            if i != j:
                distances[i][j] = minEdgeBFS(graph,i,j)
    return distances
                
    
max_weak_component_subgraph = subgraph_induced_by_nodes(data,max_weak_connectivity_component)
print(max_weak_component_subgraph)
distances = distances(max_weak_component_subgraph)
print(distances)

{187798: [303979], 303979: [71, 162316, 46204, 60070, 187798, 197012, 228499, 374743, 473024, 522020, 523892, 524207, 596972, 604497, 619588, 657764, 705695], 71: [60070, 162316, 167423, 186908, 197012, 228499, 287762, 303979, 356332, 374743, 383195, 402157, 522020, 524207, 596972, 604497, 619588, 657764, 668223, 698533, 705695, 856127], 162316: [71, 167423, 197012, 238235, 287762, 303979, 523892, 544331, 577293, 596972, 602900, 619588, 638173, 698533, 811783, 657764], 46204: [303979, 383195, 522020], 60070: [71, 228499, 303979, 374743, 522020, 524207], 197012: [71, 162316, 167423, 287762, 461469, 498253, 544331, 577293, 596972, 619588, 678328, 303979, 657764], 228499: [71, 55108, 60070, 62385, 183519, 186908, 477111, 522020, 524207, 566562, 596972, 604497, 619588, 705695, 775068, 802853, 817957, 856127, 910516, 287762, 303979, 374743, 668223], 374743: [71, 303979, 57139, 60070, 228499, 229596, 383195, 444609, 475298, 508060, 522020, 524207, 596972, 604497, 619588, 698994, 705695, 7577

KeyError: 910516